In [27]:
%%capture
from __future__ import division
import numpy as np
import pandas as pd
import scipy.stats as st
import itertools
import math
from collections import Counter, defaultdict
%load_ext autoreload
%autoreload 2

import json
import glob

In [125]:
%%capture
import matplotlib as mpl
mpl.use("Agg")
import matplotlib.pylab as plt
#%matplotlib notebook
%matplotlib inline
%load_ext base16_mplrc
%base16_mplrc light default
plt.rcParams['figure.figsize'] = (16.0, 10.0)

In [126]:
sample_data_dir = './corpus_1_pct_sample/'
test_file = sample_data_dir + 'part-00000'

In [40]:
if sampled_corpus_raw_df.empty:
    print('yes')

yes


In [70]:
sorted(list(first_test_df.columns))

['authors',
 'bodyText',
 'citationContexts',
 'citedBy',
 'citedByYearHistogram',
 'classification',
 'clusterInfo',
 'dataSetNames',
 'earliestAcquisitionDate',
 'estimatedCitationAcceleration',
 'estimatedCitationVelocity',
 'followOn',
 'id',
 'indexInclusionReason',
 'journal',
 'keyCitedByYearHistogram',
 'keyPhrases',
 'numCitedBy',
 'numCitedByEstimate',
 'numCiting',
 'numKeyCitations',
 'numKeyReferences',
 'paperAbstract',
 'presentationIds',
 'socialLinks',
 'sourceInfo',
 'title',
 'venue',
 'viewablePdfId',
 'year']

In [110]:
corpus_sample_raw_df = pd.DataFrame()

In [120]:
for file in glob.glob(sample_data_dir + '*'):
    running_idx = 0
    with open(file, 'r') as f:
        json_lines = f.readlines()
        record_by_index = {running_idx + idx: json.loads(record) for idx, record in enumerate(json_lines)}
        running_idx += len(json_lines)
        if sampled_corpus_raw_df.empty:
            corpus_sample_raw_df = pd.DataFrame.from_dict(record_by_index, orient='index')
        else:
            corpus_sample_raw_df = corpus_sample_raw_df.append(pd.DataFrame.from_dict(record_by_index, orient='index'))

In [121]:
corpus_sample_raw_df.shape

(50601, 30)

In [117]:
corpus_sample_raw_df.tail()

,authors,bodyText,citationContexts,citedBy,citedByYearHistogram,classification,clusterInfo,dataSetNames,earliestAcquisitionDate,estimatedCitationAcceleration,...,numKeyCitations,numKeyReferences,paperAbstract,presentationIds,socialLinks,sourceInfo,title,venue,viewablePdfId,year
487,"[{'ids': [], 'name': 'F H Guenther'}]",1\n\nConvex region targets lead directly to ex...,"[{'sectionName': 'Tongue body height', 'citeEn...","[52fa385e09fdc501775221505cb2b0a5bcf1a051, 18c...","[[1996, 1], [1997, 1], [1998, 1], [1999, 1], [...",{'survey': 0.0},{'duplicateIds': ['b22a3a36ea7e238b57f442365ef...,[],2014-10-01T00:00:00Z,"{'estimate': {'value': 0.0, 'confidence': 1.0,...",...,2,19,This article describes a neural network model ...,[],[],{'pdfUris': ['http://www.cns.bu.edu/~guenther/...,"Speech sound acquisition, coarticulation, and ...",Psychological review,94f16407d532cf719a54cefed26135c92d0fd007,1995.0
488,"[{'ids': ['3258268'], 'name': 'Xiaoying Zhou'}...",Introduction\n\nGastric cancer is the fourth m...,[],"[9898703db313984cc9e36142ae7f053470ecd4c1, e17...","[[2015, 5], [2016, 1]]",{'survey': 0.0},{'duplicateIds': ['120f76e0d73d08135495eeed779...,[],NaN,"{'estimate': {'value': 0.0, 'confidence': 1.0,...",...,0,0,Background: The molecular mechanism between He...,[],[],"{'pdfUris': [], 'sourceIds': [{'id': '24984017...",Decreased miR-204 in H. pylori-Associated Gast...,PloS one,f1ac2ebb273c47f532bc684cf0045c7025541833,2014.0
489,"[{'ids': ['2795873'], 'name': 'William A. Bogl...",Cyclically presented groups\n\nA celebrated th...,"[{'sectionName': '−1', 'citeEnd': 102, 'citeSt...",[eceb67b3451e45cd0eed0df12de0acbcae206044],"[[2015, 1]]",{'survey': 0.0},{'duplicateIds': []},[],2016-03-04T08:47:34.539Z,"{'estimate': {'value': 0.0, 'confidence': 1.0,...",...,0,2,,[],[],{'pdfUris': ['http://people.oregonstate.edu/~b...,On shift dynamics for cyclically presented groups,,41008690f1e66a13714688396f6aa6d1095fb0bc,2014.0
490,"[{'ids': ['2932748'], 'name': 'Fredrik Berggre...",,[],[63d9bd4e8a0f12530fda14e69d32dbcbe442718f],"[[2010, 1]]",{'survey': 0.0},{'duplicateIds': []},[],NaN,"{'estimate': {'value': 0.0, 'confidence': 1.0,...",...,1,0,,[],[],{'pdfUris': ['http://dx.doi.org/10.1109/WCNC.2...,A Non-Hierarchical Cell Search Scheme,WCNC,NaN,2007.0
491,"[{'ids': ['2869491'], 'name': 'Philipp Tiefenb...",,[],[b5432d923559ac46510399484e68c4289ea6e72b],"[[2014, 1]]",{'survey': 0.0},{'duplicateIds': []},[],NaN,"{'estimate': {'value': 0.0, 'confidence': 1.0,...",...,0,0,,[],[],{'pdfUris': ['http://dx.doi.org/10.1007/978-3-...,Augmented Reality Evaluation: A Concept Utiliz...,HCI,NaN,2014.0


In [134]:
len(pd.unique(corpus_sample_raw_df['venue']))

6259

In [132]:
corpus_sample_raw_df['venue'].value_counts()

                                                                                                                          5254
PloS one                                                                                                                  1405
ArXiv                                                                                                                      666
ICASSP                                                                                                                     321
Scientific reports                                                                                                         214
ICC                                                                                                                        206
ICIP                                                                                                                       203
IEICET                                                                                                         

In [135]:
corpus_sample_raw_df.shape

(50601, 30)